# Gradient (Steepest) Descent/Ascent

0. Fonksiyonun bir optimum noktasının olup olmadığını Hessian kullanarak kontrol et
    1. $\lvert H \rvert = \frac{\partial^2 f}{\partial x^2}\frac{\partial^2 f}{\partial y^2} - \left(\frac{\partial^2 f}{\partial x \partial y}\right)^2$
    2.  $\lvert H \rvert > 0$ ise optimum var aksi durumda eğer noktası
        1. $\frac{\partial^2 f}{\partial x^2} > 0$ ise aranan nokta bir minimum
        2. $\frac{\partial^2 f}{\partial x^2} < 0$ ise aranan nokta bir maximum
1. Bir başlangıç noktası seç ($x_0$)
2. $∇f(x_k)$ gradyan değerini hesaplayarak arama yönünü bul ($k$ burada tekrar indisi)
3. Gradyanı minimum için negatif, maximum için pozitif olarak kullan
4. Arama yönünde yeni noktayı bul: $x_{k+1}=x_k+\alpha_k∓∇f(x_k)$ 
5. $\alpha_k$ (adım büyüklüğü) değerini hesaplamak için tek değişkenli optimizasyondan faydalan

In [1]:
import numpy as np
import sympy as sp
import pandas as pd

In [2]:
x,y,h=sp.symbols('x y h')
a,b,c,d=.76,.28,-.41,-.53
f = a*x*y+b*y-c*x**2-d*y**2
f

0.41*x**2 + 0.76*x*y + 0.53*y**2 + 0.28*y

In [3]:
# hessian değerini hesapla ve optimumluğu kontrol et
d2x=sp.diff(f,x,2)
H=d2x*sp.diff(f,y,2)-(sp.diff(sp.diff(f,x),y))**2
if H>0:
    if d2x>0: 
        op='min'
        print('Hessian: %.2f ve d2f/dx2: %.2f. Minimum noktası var'%(H,d2x))
    else: 
        print('Hessian: %.2f ve d2f/dx2: %.2f. Maximum noktası var'%(H,d2x))
        op='max'

Hessian: 0.29 ve d2f/dx2: 0.82. Minimum noktası var


In [4]:
#başlangıç değerleri
V=[.5,1]
# gradyan hesapla
Gr=sp.Matrix([sp.diff(f,x),sp.diff(f,y)])
Gr

Matrix([
[       0.82*x + 0.76*y],
[0.76*x + 1.06*y + 0.28]])

In [5]:
r=2 # sonuçları yuvarlaman için
j=1 # tekrarları sayman için

In [6]:
'''Adımların daha iyi anlaşılması için döngü kurulmadı.
   Her yeni tekrar için buradan başlayarak kalan tüm hücrelerin çalıştırılması gerek'''
G=Gr.subs({x:V[0],y:V[1]})
if op=='max': g=f.subs({x:V[0]+G[0]*h,y:V[1]+G[1]*h})
elif op=='min': g=f.subs({x:V[0]-G[0]*h,y:V[1]-G[1]*h})
gs=g.simplify()
gop=sp.diff(g,h)
gops=gop.simplify()
oph=sp.solve(gop,h)[0].evalf()
if op=='max': V1=[V[0]+G[0]*oph,V[1]+G[1]*oph]
elif op=='min':  V1=[V[0]-G[0]*oph,V[1]-G[1]*oph]
V=V1

In [7]:
[G[0].evalf(),G[0].evalf()]

[1.17000000000000, 1.17000000000000]

In [8]:
g

-0.4816*h + 0.561249*(0.427350427350427 - h)**2 + 0.76*(0.5 - 1.17*h)*(1 - 1.72*h) + 1.567952*(0.581395348837209 - h)**2 + 0.28

In [9]:
sp.Poly([round(i,r) for i in sp.Poly(g.simplify(),h).all_coeffs()], h).as_expr()

3.66*h**2 - 4.33*h + 1.29

In [10]:
gop

7.31725*h - 4.3273

In [11]:
sp.Poly([round(i,r) for i in sp.Poly(gop.simplify(), h).all_coeffs()], h).as_expr()

7.32*h - 4.33

In [12]:
oph.round(r)

0.59

In [13]:
V

[-0.191918548635075, -0.0171794048310501]

In [14]:
f.subs({x:V[0],y:V[1]}).round(3)

0.013

In [15]:
print(j,'. tekrar tamamlandı')
j=j+1

1 . tekrar tamamlandı
